In [1]:
import json 

# loadijg dataset
dataset = json.load(open("Dataset/tf-idf-dataset.json", "r"))

In [3]:
import numpy as np

from sklearn.model_selection import train_test_split


X = []
y = []
classes = dataset["Class_Names"]

# extracting X and y from json dataset
for data in dataset["Dataset"]:
    X.append(data["x"])
    y.append(data["y"])

print(f"total dataset length: {len(X)}")

X = np.array(X)
y = np.array(y)

# spliting dataset
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


total dataset length: 42663


In [6]:
print(x_train.shape)
print(y_train.shape)

(29864, 5000)
(29864, 8)


In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
from keras import backend as K

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [10]:
## training ANN with keras

from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation


model = Sequential()
model.add(Dense(1000, input_shape=(5000,)))
model.add(Activation('relu'))
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dense(len(classes))) # output layer
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', recall, precision])
print(model.summary())

model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=128, epochs=10, verbose=1)
model.save(f"Models/keras_ann_v1.h5")


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 1000)              5001000   
                                                                 
 activation_8 (Activation)   (None, 1000)              0         
                                                                 
 dense_9 (Dense)             (None, 500)               500500    
                                                                 
 activation_9 (Activation)   (None, 500)               0         
                                                                 
 dense_10 (Dense)            (None, 200)               100200    
                                                                 
 activation_10 (Activation)  (None, 200)               0         
                                                                 
 dense_11 (Dense)            (None, 8)                

In [11]:
# getting y_red for x_test i want to see test set precission, classification_report and confusion_matrix
y_pred = model.predict(x_test)


In [14]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, classification_report

# np.argmax() gets max argument index of y list
# example argmax to [0, 0, 1, 0] result will be 2 (class index) 
test_accuracy_score = accuracy_score(np.argmax(y_test), np.argmax(y_pred, axis=1))
print(f"Accuracy for x_test: {accuracy_score}")
# print(confusion_matrix())
# print(f1_score())

TypeError: Singleton array 6 cannot be considered a valid collection.